In [18]:
import os
import codecs

In [19]:
PATH = os.getcwd()
dir_path = PATH + '/cg3-files'

output_path = PATH + '/conll-files'

In [20]:
def get_next_symbol(lines):
    counter = 0
    for l in lines:
        if len(l.rstrip().lstrip()) > 0:
            if l.startswith('>"') or l.startswith('"<'):
                return counter
            else:
                counter += 1
    return counter
        
    

In [21]:
def get_sentence(lines):
    sentence = ''
    for l in lines:
        if len(l.rstrip().lstrip()) > 0:
            if l.startswith('>"') or l.startswith('"<'):
                sentence += l.replace('>"', '').replace('"<', '').replace('\n', '')+' '
        else: 
            return sentence
    return sentence
                                                    
                
            
    

In [28]:
def format_text(lines, text, line, symb_count, sid, sentence, sentence_counter):

    if len(lines) >= 1:  
        if len(lines[0].strip()) == 0:
            sentence_counter += 1
            text += '\n#sent_id = ' + str(sentence_counter) + '\n'
            sentencex = get_sentence(lines[1:])
            text += '#text: ' + sentencex + '\n'
            
            sentence = ''
            return format_text(lines[1:], text+'\n', line, symb_count, 1, sentence, sentence_counter)
        
        symb_count = get_next_symbol(lines)   
        #print(symb_count, ' ' , lines)
        if symb_count == 0:
            Nsymb_count = get_next_symbol(lines[1:])
            if Nsymb_count > 1:
                startIdx = sid
                endIdx = sid + Nsymb_count - 1
                line = str(startIdx) + '-' + str(endIdx) + '\t' +lines[0].strip()[2:-2]+'\n'
                text += line
                sentence += lines[0].strip()[2:-2]+' '
                return format_text(lines[1:], text, line, 1, sid, sentence, sentence_counter)
            else:
                line = str(sid) + '\t' +lines[0].strip()[2:-2] + '\t'
                sentence += lines[0].strip()[2:-2]+' '
                text += line
                
                return format_text(lines[1:], text, line, symb_count, sid, sentence, sentence_counter)
            
        elif symb_count == 1:            
            if lines[0].startswith('#'):
                sentence_counter += 1
                text += '#sent_id = ' + str(sentence_counter) + '\n'
                sentencex = get_sentence(lines[1:])
                text += '#text: ' + sentencex + '\n'
                text += line
                return format_text(lines[1:], text, line, symb_count, 1, sentence, sentence_counter)
            else:
                w_pos = lines[0].rstrip().lstrip().split(' ')
                w = w_pos[0].replace('"','')
                if len(w_pos) > 1:
                    xpos = w_pos[1].upper()
                    upos = "|".join(w_pos[1:]).upper()
                else:
                    xpos = w_pos[0].upper()
                    upos = "|".join(w_pos[0:]).upper()
                        
                line = '\t' + w + '\t' + xpos + '\t' + xpos + '\t' + upos + '\t'+ str(sid) + '\t _ \t _ \t _\n'
                text += line
                sid += 1
            
            return format_text(lines[1:], text, line, symb_count, sid, sentence, sentence_counter)
      
        elif symb_count > 1:
            for j in range(0, symb_count):
                if len(lines[j].rstrip().lstrip()) > 0:
                    w_pos = lines[j].strip().split(' ')
                    w = w_pos[0].replace('"', '')            
                    if len(w_pos) > 1:
                        xpos = w_pos[1].upper()
                        upos = "|".join(w_pos[1:]).upper()
                    else:
                        xpos = w_pos[0].upper()
                        upos = "|".join(w_pos[0:]).upper()
                        
                    line = str(sid) + '\t' + w + '\t' + xpos + '\t' + xpos + '\t' + upos + '\t' + str(sid) + '\t _ \t _ \t _ \n'
                    text += line
                    sid += 1      
            return format_text(lines[symb_count:], text, line, symb_count, sid, sentence, sentence_counter)
        
        else:
            return format_text(lines[1:], text, line, symb_count, sid, sentence, sentence_counter)

    return text, symb_count, sentence

#lines = ["##sfd", '"<this>"', "is", "a", "sentence",  '\n', '"<a>"',"new", "finally",'"<endxxx>"', "end", "\n"]
#text, symb_count, sentence = format_text(lines, text='', line='', symb_count=0, sid=1, sentence='', sentence_counter=1)
#print('text: \n%s\nsymbol count:%s\ni: %d\nSentence:\n%s:' %(text, symb_count, i, sentence))

In [29]:
for files in os.listdir(dir_path):
    #print(files)
    output = open(output_path + '/'+files, 'w')
    file_content = dir_path+'/'+files
    with open(file_content, 'r') as f:
        lines = f.readlines()
        t = format_text(lines, text='', line='', symb_count=1, sid=1, sentence='', sentence_counter=0)
        output.write(t[0])
    output.close()
